<a href="https://colab.research.google.com/github/edwardj4747/Dataset-Labeling/blob/master/ML_notebooks/Prepare_ML_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Todo


*   Filter on the number of occurences of dataset to use that
*   Precision Recall Analysis
* Word Embeddings



## Load in the sentences

In [ ]:
import json
# with open('/content/sample_data/raw_data_aura_mls_ONLY_noNO.json') as f:
#   raw_data = json.load(f)

# # use the braod sentences
with open('/content/sample_data/raw_data_all_papers_broad_aura_mls_ONLY.json') as f:
  raw_data = json.load(f)

Restrict the ground_truths datasets to only ML2xx

In [ ]:
for key, value in raw_data.items():
  new_ground_truths = []
  for gt in value['ground_truths']:
    if gt.startswith("ML2"):
      new_ground_truths.append(gt)
  raw_data[key]['ground_truths'] = new_ground_truths

In [ ]:
count = 0
for k, v in raw_data.items():
  print(k, v)
  count += 1
  if count == 3:
    break

4MFNKR4V {'ground_truths': ['ML2T', 'ML2H2O', 'ML2O3'], 'data': {'(aura/mls,None)': {'mission': 'aura', 'instrument': 'mls', 'variable': 'None', 'sentences': ['mls launched in 2004 the aura satellite became the second member of the a-train array of satellites that observes the earth in a sun-synchronous polar orbit ~15 times a day'], 'num_sentences': 1}, '(aura/mls,o3)': {'mission': 'aura', 'instrument': 'mls', 'variable': 'o3', 'sentences': ['aura is equipped with the mls instrument which is designed to make high-quality measurements of upper atmospheric t h2o o3 and an assortment of other climate sensitive atmospheric constituents'], 'num_sentences': 1}, '(aura/mls,h2o)': {'mission': 'aura', 'instrument': 'mls', 'variable': 'h2o', 'sentences': ['aura is equipped with the mls instrument which is designed to make high-quality measurements of upper atmospheric t h2o o3 and an assortment of other climate sensitive atmospheric constituents'], 'num_sentences': 1}, '(aura/mls,t)': {'mission

Clean text function

In [ ]:
def clean(sentence):
  # "n a s a a u t h o r m a n u s c r i p t n a s a a u t h o r m a n u s c r i p t n a s a a u t h o r m a n u s c r i p t o3 data assimilated in merra-2 sbuv nimbus-7 sbuv 2 noaa-11 sbuv 2 noaa-14 sbuv 2 noaa-16 sbuv 2 noaa-17 omi eos aura mls eos aura 2 statistical comparisons of the merra-2 total o3 against toms data",
  # "20 mls avk\u2212smoothed somora 2 4 6 o3 vmr 8 2\u2212540 \u221220 0 20 40 2\u221251 0 1 aura mls\u2212somora aura mls\u2212somora fig",
  sentence = re.sub(r'[=~,:%;\[\]\(\)]', '', sentence)
  sentence = re.sub(r'--', ' ', sentence)
  sentence = re.sub(r'[^\-a-zA-Z][0-9]+', '', sentence) # numbers
  sentence = re.sub(r'^[0-9]+ ', '', sentence) # numbers at beginning
  sentence = re.sub(r'^([a-z] ){2,}', '', sentence)  # n a s a a u t h o ...

  sentence = re.sub(r' {2,}', ' ', sentence)  # extra spaces

  stop_words = set(stopwords.words('english'))
  stop_words.remove('t') 
  word_tokens = word_tokenize(sentence)  # look at tokenization of things like 'sage ii' vs 'sage-ii'  
  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  sentence = ' '.join(filtered_sentence)
  sentence = re.sub(r' \.', '.', sentence)
  return sentence

In [ ]:
def clean(sentence):
    sentence = re.sub(r'\(\w+ et al., [0-9]{2,4}\)''', '', sentence)  # (Author et al., 2015)
    sentence = re.sub(r'et al.', 'et al', sentence)  # remove periods from 'et al' for better sentence spliting
    sentence = re.sub(r'e.g.', 'eg', sentence)
    sentence = re.sub(r'i.e.', 'ie', sentence)

    sentence = re.sub(r'[^\x00-\x7F]+', '', sentence)  # non ascii characters
    sentence = re.sub(r'https?://.*?((.com)|(.org)|(.gov))', '', sentence)  # links http://disc.sci.gsfc.nasa.gov

    sentence = re.sub(r'[=~,:%;<>/\[\]\(\)\']', '', sentence)
    sentence = re.sub(r'--', ' ', sentence)
    sentence = re.sub(r'[^\-a-zA-Z][0-9]+', '', sentence)  # numbers
    sentence = re.sub(r'^[0-9]+ ', '', sentence)  # numbers at beginning
    sentence = re.sub(r'^([a-z] ){2,}', '', sentence)  # n a s a a u t h o ...

    sentence = re.sub(r' {2,}', ' ', sentence)  # extra spaces

    stop_words = set(stopwords.words('english'))
    stop_words.remove('t')
    word_tokens = word_tokenize(sentence)  # look at tokenization of things like 'sage ii' vs 'sage-ii'
    filtered_sentence = [w for w in word_tokens if not w in stop_words]

    sentence = ' '.join(filtered_sentence)
    sentence = re.sub(r' \.', '.', sentence)
    return sentence

In [ ]:
clean("aura is equipped with the mls instrument which is designed to make high-quality measurements of upper atmospheric t h2o o3 and an assortment of other climate sensitive atmospheric constituents")

'aura equipped mls instrument designed make high-quality measurements upper atmospheric t h2o o3 assortment climate sensitieatmospheric constituents'

## Dataset Mapping

Add in a threshold for the number of occurrences of ML2xx datasets

In [ ]:
# get all ML2xx datasets
all_datasets = set()
for key, value in raw_data.items():
  for gt in value['ground_truths']:
    all_datasets.add(gt)

In [ ]:
threshold = 10
counts = {}
for key, value in raw_data.items():
  for gt in value['ground_truths']:
    if gt.startswith("ML2"):
      counts[gt] = counts.get(gt, 0) + 1

all_datasets_above_threshold = set()
for key, value in counts.items():
  if value >= threshold:
    all_datasets_above_threshold.add(key)

counts, all_datasets_above_threshold

({'ML2BRO': 3,
  'ML2CH3CL': 4,
  'ML2CH3CN': 2,
  'ML2CH3OH': 2,
  'ML2CLO': 21,
  'ML2CO': 36,
  'ML2DGG': 1,
  'ML2GPH': 6,
  'ML2H2O': 69,
  'ML2H2O_NRT': 1,
  'ML2HCL': 27,
  'ML2HCN': 3,
  'ML2HNO3': 39,
  'ML2HNO3_NRT': 1,
  'ML2HO2': 6,
  'ML2HOCL': 3,
  'ML2IWC': 18,
  'ML2N2O': 24,
  'ML2O3': 115,
  'ML2OH': 9,
  'ML2RHI': 6,
  'ML2SO2': 5,
  'ML2T': 34},
 {'ML2CLO',
  'ML2CO',
  'ML2H2O',
  'ML2HCL',
  'ML2HNO3',
  'ML2IWC',
  'ML2N2O',
  'ML2O3',
  'ML2T'})

In [ ]:
# if want to create a mapping that does not include all datasets
all_datasets = all_datasets_above_threshold

In [ ]:
dataset_mapping = {dataset: i for i, dataset in enumerate(all_datasets)}
dataset_mapping

{'ML2CLO': 1,
 'ML2CO': 0,
 'ML2H2O': 3,
 'ML2HCL': 2,
 'ML2HNO3': 5,
 'ML2IWC': 8,
 'ML2N2O': 4,
 'ML2O3': 7,
 'ML2T': 6}

###### save the dataset mapping

In [ ]:
# save the dataset_mapping
threshold = str(threshold)
dataset_mapping_file_name = str(len(dataset_mapping)) + "_dataset_mapping_threshold_" + threshold

with open('ml_ready_data/' + dataset_mapping_file_name + '.json', 'w', encoding='utf-8') as f:
  json.dump(dataset_mapping, f, indent=4)

dataset_mapping_file_name

'9_dataset_mapping_threshold_10'

## For each paper create one sentence and the datasets_used vector

In [ ]:
num_papers = len(raw_data)
num_datasets = len(dataset_mapping)
num_papers, num_datasets

(247, 9)

In [ ]:
papers_sentences = []  # one sentence for each paper
dataset_labels = np.zeros( (num_papers, num_datasets))
dataset_labels_list = []
rows_to_delete = []

for index, (key, value) in enumerate(raw_data.items()):
  # add the sentences
  document_sentences = []
  for value_dict in value['data'].values():
    document_sentences += value_dict['sentences']
  if len(document_sentences) == 0:   # ignore the papers without any sentences
    rows_to_delete.append(index)
    continue
 
  unique_document_sentences = list(set(document_sentences))
  joined_sentences = '. '.join(unique_document_sentences)
  clean_sentences = clean(joined_sentences)
  papers_sentences.append(clean_sentences)


  # add in the datasets
  for gt in value['ground_truths']:
    if dataset_mapping.get(gt, None) is not None:  # some keys may not have values b/c didn't meet threshold
      dataset_labels[index, dataset_mapping[gt]] = 1

dataset_labels = np.delete(dataset_labels, rows_to_delete, 0)

papers_sentences[0], dataset_labels[0, :]

('since mls o3 dietly assimilated merra2 o3 mixing ratios close hybrid profie three arm sie. mls h2o retrieved frequency ghz acceptable range of-0. intercomparison studies suggest o3 values mls match fairly well multiinstrument means sage ii values. upper troposphere estimated mls h2o uncertainty tropics midlatitudes high latitudes. mls o3 product used study also version. recommended range mls o3 product. black lierepresents hybrid profie arm-megd soundings mls atmospheric t h2o. estimated uncertainty mls h2o stratosphere. stratospheric o3 mls retrieved frequency ghz offers best precision wievertical range. aura equipped mls instrument designed make high-quality measurements upper atmospheric t h2o o3 assortment climate sensitieatmospheric constituents. o3 profieis airs mls. retrieved mls t used study version. mls launched aura satelliebecame second member a-train array satellie observes earth sun-synchronous polar orbit tie day',
 array([0., 0., 0., 1., 0., 0., 1., 1., 0.]))

In [ ]:
len(papers_sentences), dataset_labels.shape

(234, (234, 9))

In [ ]:
papers_sentences

['since mls o3 dietly assimilated merra2 o3 mixing ratios close hybrid profie three arm sie. mls h2o retrieved frequency ghz acceptable range of-0. intercomparison studies suggest o3 values mls match fairly well multiinstrument means sage ii values. upper troposphere estimated mls h2o uncertainty tropics midlatitudes high latitudes. mls o3 product used study also version. recommended range mls o3 product. black lierepresents hybrid profie arm-megd soundings mls atmospheric t h2o. estimated uncertainty mls h2o stratosphere. stratospheric o3 mls retrieved frequency ghz offers best precision wievertical range. aura equipped mls instrument designed make high-quality measurements upper atmospheric t h2o o3 assortment climate sensitieatmospheric constituents. o3 profieis airs mls. retrieved mls t used study version. mls launched aura satelliebecame second member a-train array satellie observes earth sun-synchronous polar orbit tie day',
 'aggregate results individual h2o retrievals mls revea

Tokenize & one-hot the sentences

In [ ]:
from keras.preprocessing.text import Tokenizer

docs = papers_sentences

# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)
# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(docs, mode='tfidf')  # modes binary, count, tfidf, freq
print(encoded_docs)

OrderedDict([('mls', 4631), ('o3', 1679), ('product', 95), ('used', 233), ('study', 115), ('also', 225), ('version', 108), ('launched', 63), ('aura', 1409), ('satellite', 325), ('became', 2), ('second', 18), ('member', 1), ('a', 44), ('train', 14), ('array', 1), ('satellites', 10), ('observes', 16), ('earth', 95), ('sun', 25), ('synchronous', 23), ('polar', 112), ('orbit', 74), ('times', 16), ('day', 108), ('recommended', 15), ('range', 85), ('since', 79), ('directly', 12), ('assimilated', 58), ('merra2', 22), ('mixing', 226), ('ratios', 109), ('close', 24), ('hybrid', 2), ('profiles', 427), ('three', 55), ('arm', 2), ('sites', 8), ('black', 25), ('line', 55), ('represents', 15), ('merged', 52), ('soundings', 2), ('atmospheric', 95), ('t', 416), ('h2o', 693), ('profile', 160), ('airs', 110), ('estimated', 66), ('uncertainty', 47), ('stratosphere', 207), ('retrieved', 74), ('frequency', 18), ('ghz', 57), ('acceptable', 2), ('of', 24), ('0', 9), ('stratospheric', 221), ('offers', 6), ('b

### Prepare training vectors for doc2Vec

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
len(papers_sentences)
train_test_percentage = 0.2
train_test_split = int(train_test_percentage * len(papers_sentences))
train_test_split

46

In [ ]:
train_papers = papers_sentences[train_test_split:]
tagged_train_data = []

for i, sentence in enumerate(train_papers):
  tagged_doc = TaggedDocument(words=sentence, tags=[str(i)])
  tagged_train_data.append(tagged_doc)

In [ ]:
len(train_papers), len(tagged_train_data)

(188, 188)

In [ ]:
index = 0
train_papers[index], tagged_train_data[index]

('compare waccm calculations mls observations showing waccm hno3 lower. vertical profiles hno3 mixing ratios waccm balloon-borne measurements mls satellite. figure shows waccm predicted hno3 compares well improved limb atmospheric spectrometer mls satellite measurements outside spe periods',
 TaggedDocument(words='compare waccm calculations mls observations showing waccm hno3 lower. vertical profiles hno3 mixing ratios waccm balloon-borne measurements mls satellite. figure shows waccm predicted hno3 compares well improved limb atmospheric spectrometer mls satellite measurements outside spe periods', tags=['0']))

In [ ]:
# ideally want to try training the Doc2Vec on just the full text of the papers

### Create model for Doc2Vec

In [ ]:
model = None

In [ ]:
vector_size = 32
epochs = 50
model = Doc2Vec(vector_size=vector_size, min_count=1, epochs=epochs, seed=1, workers=1)

In [ ]:
model.build_vocab(tagged_train_data)

In [ ]:
model.train(tagged_train_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.total_train_time

5.373223634986971

In [ ]:
# sanity check. Check how many inferred vectors are most similar to actual one
count = 0
for i in range(len(tagged_train_data)):
  inferred_vector = model.infer_vector(tagged_train_data[i].words)
  most_similar = model.docvecs.most_similar([inferred_vector], topn=1)[0][0]
  # print(most_similar)
  if most_similar == str(i):
    count += 1

print(count, '/', len(tagged_train_data))
print(count/len(tagged_train_data))

188 / 188
1.0


Use the model to infer word vectors for all of the original sentences and then save them

In [ ]:
encoded_docs = np.zeros( (len(papers_sentences), vector_size) )
encoded_docs.shape, len(papers_sentences)

((234, 32), 234)

In [ ]:
# the first train_test_split papers will be unseen and totally inferred
# the remainder of the papers were in the training set so they have been seen
for i in range(len(papers_sentences)):
  # inferred_vector = model.infer_vector(tagged_train_data[i].words)
  inferred_vector = model.infer_vector(papers_sentences[i])
  encoded_docs[i, :] = inferred_vector

encoded_docs

array([[-0.25602606, -0.67229378,  0.18899873, ...,  0.21936288,
         0.09616316, -0.68966091],
       [-0.06930058, -0.15150073, -0.38204777, ..., -0.23781285,
        -0.30828115, -1.64354062],
       [ 0.20663281, -1.28198349,  1.07013309, ..., -0.61890596,
         0.76648194,  1.30882287],
       ...,
       [ 0.09334937, -0.22381939, -0.38237026, ...,  0.251775  ,
        -0.06819416, -0.55283856],
       [ 0.61810654,  1.2663691 ,  0.11712242, ...,  0.12567824,
         1.43909323, -0.09343802],
       [ 0.50245416,  0.69061488,  0.30427369, ..., -0.24310972,
         0.17648947,  0.18081786]])

In [ ]:
index = 46
papers_sentences[index], tagged_train_data[index - train_test_split]

('compare waccm calculations mls observations showing waccm hno3 lower. vertical profiles hno3 mixing ratios waccm balloon-borne measurements mls satellite. figure shows waccm predicted hno3 compares well improved limb atmospheric spectrometer mls satellite measurements outside spe periods',
 TaggedDocument(words='compare waccm calculations mls observations showing waccm hno3 lower. vertical profiles hno3 mixing ratios waccm balloon-borne measurements mls satellite. figure shows waccm predicted hno3 compares well improved limb atmospheric spectrometer mls satellite measurements outside spe periods', tags=['0']))

In [ ]:
# Note the results will not be exactly the same because algo is not deterministic as PYTHONHASHSEED is not set for environment
s = "mls observations showing waccm hno3 lower. vertical profiles hno3"
model.infer_vector(s), model.infer_vector(s)

(array([ 0.1189685 ,  0.23656537, -0.07851534, -0.21577246,  0.7903635 ,
         0.0810025 ,  0.6126154 , -0.04115227, -0.58528304, -0.047262  ,
        -0.06807674, -0.34082907,  0.1736035 ,  0.2134969 , -0.5242941 ,
        -0.69322467, -0.06225577,  0.364145  , -0.12325693, -0.546118  ,
         0.2543721 ,  0.21459264, -0.08470464, -0.35826257, -0.0424878 ,
        -0.13992934,  0.8646181 ,  0.19111434, -0.1257885 ,  0.15196948,
        -0.5739112 , -0.06516577], dtype=float32),
 array([ 0.20859694,  0.18499735,  0.06143554, -0.20228085,  0.87139857,
         0.09707559,  0.87242764, -0.01822628, -0.5436295 ,  0.01758441,
        -0.09656944, -0.33579588,  0.08295052,  0.1525227 , -0.4969917 ,
        -0.6752688 , -0.04414976,  0.34909037, -0.29487652, -0.51743025,
         0.23946469,  0.33222955, -0.0444658 , -0.27272168, -0.03683008,
        -0.06158718,  0.8657869 ,  0.15172775,  0.0171466 ,  0.15338957,
        -0.56100976, -0.04070149], dtype=float32))

### Doc2Vec with an already trained moel

In [ ]:
model = Doc2Vec.load('/content/sample_data/vs32e100_200k_sentences')

In [ ]:
vs = 32
vector_size=vs
epochs = 100
len(papers_sentences)

234

In [ ]:
encoded_docs = np.zeros((len(papers_sentences), vs))
encoded_docs.shape

(234, 32)

In [ ]:
for i in range(len(papers_sentences)):
  encoded_docs[i, :] = model.infer_vector(papers_sentences[i], steps=10000)  # hopefully this many steps makes it somewhat-ish deterministic

In [ ]:
encoded_docs

array([[ 0.52622336, -1.2161957 , -0.3819215 , ...,  0.417348  ,
         1.41460371,  0.6425463 ],
       [ 2.01511788, -0.78863031, -1.30535448, ..., -1.13700116,
        -0.03659254, -1.97091615],
       [-4.8999052 ,  0.61206526, -0.23863694, ..., -1.22970462,
        -1.42693436,  5.5723896 ],
       ...,
       [-4.82347298,  0.96647578,  3.57008028, ..., -4.52962017,
        -8.33773804, -0.88316011],
       [ 0.60638726,  4.97212219,  1.13236606, ..., -4.43715048,
        -3.05120587, -0.34968385],
       [ 0.60185695,  4.91944599, -0.938501  , ..., -0.83469599,
        -3.92474627,  3.03522897]])

### Save the Data


In [ ]:
encoded_docs.shape, dataset_labels.shape

((234, 32), (234, 9))

In [ ]:
# save the X data
base_name = "tfidf_"
broad_or_strict = "braod" 
x_file_name = base_name + str(encoded_docs.shape[1]) + "_features_" + broad_or_strict + "_" + str(encoded_docs.shape[0]) + "_papers"
x_file_name

'tfidf_32_features_braod_234_papers'

In [ ]:
# save the X data FOR Doc2Vec
base_name = "doc2vec_ALL_CORPUS_"
broad_or_strict = "broad" 
x_file_name = base_name + broad_or_strict + "_" + str(encoded_docs.shape[0]) + "_papers_VS_" + str(vector_size) + "_epochs_" + str(epochs)
x_file_name

'doc2vec_ALL_CORPUS_broad_234_papers_VS_32_epochs_100'

In [ ]:
with open('/content/ml_ready_data/' + x_file_name + '.npy', 'wb') as f:
  np.save(f, encoded_docs)

In [ ]:
# save the y data
dataset_labels.shape
y_file_name = "ML2xx_num_datasets_" + str(dataset_labels.shape[1]) + "_papers_" + str(dataset_labels.shape[0])
y_file_name

'ML2xx_num_datasets_9_papers_160'

In [ ]:
with open('/content/ml_ready_data/' + y_file_name + '.npy', 'wb') as f:
  np.save(f, dataset_labels)

#### Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
len(encoded_docs), dataset_labels.shape

(234, (234, 23))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(encoded_docs, dataset_labels, test_size=0.2, random_state=1)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((187, 5182), (47, 5182), (187, 23), (47, 23))

In [ ]:
dtc = DecisionTreeClassifier(max_depth=8)

In [ ]:
dtc.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
dtc.score(x_train, y_train)

0.8021390374331551

In [ ]:
dtc.score(x_test, y_test)

0.46808510638297873

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
precision_recall_fscore_support([[1], [1], [0], [1]], [[1], [1], [1], [0]], average='micro')
# tp 2
#fp 1

(0.5, 0.5, 0.5, None)

In [ ]:
precision_recall_fscore_support(y_test, dtc.predict(x_test))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.5       , 0.        , 0.        , 0.        , 1.        ,
        1.        , 0.        , 0.        , 0.        , 0.84      ,
        0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.66666667]),
 array([1.        , 0.        , 0.        , 0.        , 1.        ,
        0.625     , 0.        , 0.        , 0.        , 0.95454545,
        0.        , 0.        , 0.        , 0.        , 0.25      ,
        0.        , 0.75      , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.5       ]),
 array([0.66666667, 0.        , 0.        , 0.        , 1.        ,
        0.76923077, 0.        , 0.        , 0.        , 0.89361702,
        0.        , 0.        , 0.        , 0.        , 0.33333333,
        0.        , 0.85714286, 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.57142857]),
 array([ 1,  2,  1,  2,  3,  8,  1,  7,  0, 22

@Todo: An analysis of what this found/did not find